# Clustering Practice Notebook

In the cell below we have generated 7 different datasets for you to cluster. This exercise is based on an [sklearn publication](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html) demonstrating how different clustering techniques behave. You are welcome to import additional clustering methods, and experiment with them if you'd like. In this notebook, we will focus on KMeans and Agglomerative clustering

In [ ]:
# Run this cell unchanged
import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets
random_state = 170
np.random.seed(0)
n_samples = 1500
data = []
data.append(datasets.make_blobs(n_samples=n_samples, random_state=8)[0])
data.append(datasets.make_blobs(n_samples=n_samples, random_state=random_state)[0])
data.append(datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)[0])
data.append(datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)[0])
data.append(datasets.make_moons(n_samples=n_samples, noise=.05)[0])
data.append(np.random.rand(n_samples, 2))


X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)

data.append(X_aniso)

fig, axes = plt.subplots(2,4, figsize=(15,6))

titles = ['Tight and far', 'Tight and close', 'Uneven variance', 'Circles',
          'Moons', 'No Clusters', 'Non Circular']

for idx in range(1,8):
    ax = axes[(idx-1)//4, (idx-1)%4]
    X = data[idx-1]
    ax.scatter(X[:,0], X[:,1])
    ax.set_title(titles[idx-1], fontsize=15)
fig.tight_layout()

# Dataset # 1

In the cell below, we visualize the first dataset and assigned the data to the variable `data_1`.

In [ ]:
# Run this cell unchanged
data_1 = data[0]
plt.figure(figsize=(10,5))
plt.scatter(data_1[:,0], data_1[:,1]);

In the cell below, import the Kmeans clustering object from sklearn, and generate predictions for the dataset. Store the predictions in the variable `preds`.

>Because clustering techniques are typically relient on distance metrics, you will also need to scale!

In [ ]:
# Import the necessary sklearn objects

# Scale the data

# Initialize a Kmeans object with a selected number of clusters

# Fit the kmeans object to the data

# Generate predictions for the data


# Plot the results
plt.scatter(data_1[:,0], data_1[:,1], c=preds);

# Dataset #2

In the cell below, we visualize the second dataset and assigned the data to the variable `data_2`.

In [ ]:
# Run this cell unchanged
data_2 = data[1]
plt.figure(figsize=(10,5))
plt.scatter(data_2[:,0], data_2[:,1]);

In the cell below, apply the kmeans algorithm to dataset 2, and assign your predictions to the variable `preds`.

In [ ]:
# Scale data

# Create a kmeans object

# Fit kmeans to the data

# Generate predictions

# Plot the results
plt.scatter(data_2[:,0], data_2[:,1], c=preds);

## Finding `n_clusters`

This is all fine an dandy, but what if we do not know the number of clusters?

If we have reason to believe that the clusters of our data are structured similary to above (A sort of spherical or "blob" like shape), then the `silhouette score` often times does a good job of finding the optimal number of clusters.

Let's try it out. In the cell below, import `silhouette_score` from sklearn.

In [ ]:
# Your code here

Now we will use `np.random` to select a random number of clusters. 

In [ ]:
# Run this cell unchanged

# Randomly select an integer between 2 and 10
n_clusters = np.random.choice(range(2, 11))

Next, we will create a dataset of clusters setting the number of clusters to the randomly selected integer from the cell above.

In [ ]:
# Run this cell unchanged

# Create the dataset with an unknown # of clusters
unknown = datasets.make_blobs(n_samples=n_samples, centers=n_clusters)[0]

# Scale the data for clustering with kmeans
unknown = StandardScaler().fit_transform(unknown)

Now we will loop over a range of numbers. For this experiment we will loop over 2 to 20 with the assumption that we have absolutely no idea how many clusters are in the data. 

For each iteration, we will:
1. Fit a kmeans model to the variable `unknown` with our guess for the number of clusters
1. Generate predictions for the data
1. Pass the dataset and the predictions into `silhouette_score`
1. Append those scores to a list
1. Plot our scores for each guess.

We will also plot the elbow plot for [kmean inertia](https://stats.stackexchange.com/a/78413) which is another way of selecting n_clusters for the kmeans algorithm.

In [ ]:
# Run this cell unchanged

# Run this cell unchanged
fig, axes = plt.subplots(1, 3, figsize=(15,6))

scores = []
inertias = []
for k in range(2,20):
    # Fit a kmeans with a guess at the number of clusters
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(unknown)
    # Generate predictions
    preds = kmeans.predict(unknown)
    # Append our scores
    score = silhouette_score(unknown, preds)
    scores.append(score)
    inertias.append(kmeans.inertia_)

    
#
# Plot the silhouette scores
best_cluster = sorted(list(enumerate(scores)), key=lambda x: x[1])[-1][0] + 2
axes[0].plot(range(2,20), scores)
axes[0].set_title('Silhouette Scores')
axes[0].vlines(best_cluster, min(scores), max(scores), linestyle='--', label='Best n_clusters')
axes[0].legend()
# Plot the intertia elbow plot
axes[1].plot(range(2, 20), inertias)
axes[1].set_title('Inertias')

# Installing a python package for
# identifying the elbow of a line plot
! pip install kneed -q
from kneed import KneeLocator
elbow = KneeLocator(range(2,20),inertias,
                    curve='convex',
                    direction='decreasing',
                    interp_method='interp1d').knee
best_cluster_inertia = elbow
# Plotting a vertical line indicating the optimal n_clusters
axes[1].vlines(elbow, 0, max(inertias), linestyle='--', label='Best n_clusters')
axes[1].legend()


# Plot the clusters using the score deemed best by silhouette score
kmeans = KMeans(n_clusters=best_cluster).fit(unknown)
preds = kmeans.predict(unknown)
axes[2].scatter(unknown[:,0], unknown[:,1], c=preds)
axes[2].set_title('Model Clusters\nusing Silhouette Score')

# Print the true number of cluster vs our clusters
print('Number of true clusters:', n_clusters)
print('Best n_clusters using silhouette score:', best_cluster)
print('Best n_clusters using inertias:', best_cluster_inertia)

**Note:** This will not always work perfectly. The clusters are created randomly, and sometimes the clusters will not be obviously seperable! If you ran the code above and you ended up with a number of clusters that deviated from the true value, run  all of the code that creates this experiment a couple times, and evaluate how the arrangement of the clusters impacts the silhouette score and inertia's estimation!

# Cluster Data 3

In the cell below, we visualize the second dataset and assigned the data to the variable `data_3`.

In [ ]:
# Run this cell unchanged

data_3 = data[2]
plt.figure(figsize=(10,5))
plt.scatter(data_3[:,0], data_3[:,1]);

In the cell below, fit Kmeans to `data_3` and generate predictions. Store your predictions in the variable `preds` and plot the results!

In [ ]:
# Your code here

# Cluster data 4
In the cell below, we visualize the second dataset and assigned the data to the variable `data_4`.

In [ ]:
# Run this cell unchanged

data_4 = data[3]
plt.figure(figsize=(10,5))
plt.scatter(data_4[:,0], data_4[:,1]);

In the cell below, fit Kmeans to `data_4` and generate predictions. Store your predictions in the variable `preds` and plot the results!

In [ ]:
# Your code here

## Apply Agglomerative Clustering to Data 4

Agglomerative clustering is a form of hierarchical clustering that begins by assigning every point their own cluster, and merging clusters together until the desired number of clusters are reached.

A very significant hyperparameter for Agglomerative clustering is `linkage`.

**Here are the linkage options available in sklearn:**

`'ward'` minimizes the variance of the clusters being merged.

`'average'` uses the average of the distances of each observation of the two sets.

`'complete'` or ‘maximum’ linkage uses the maximum distances between all observations of the two sets.

`'single'` uses the minimum of the distances between all observations of the two sets.

-----

`ward` is the default linkage for sklearn's Agglomerative clustering. Let's see how it does.

In [ ]:
# Run this cell unchanged

# Import AgglomerativeClustering
from sklearn.cluster import AgglomerativeClustering

# Initialize ward clustering
ward = AgglomerativeClustering(n_clusters=2, linkage='ward')
# Fit the clustering model
ward.fit(data_4)
# Collect the predicted labels
preds = ward.labels_
# Plot the results
plt.scatter(data_4[:,0], data_4[:,1], c=preds);

Not great! For this data, the centroid of a cluster and the variance of a cluster are not particularly informative of the cluster's boundaries. (The centers for both clusters are essentially the same!) Because ward is attempting to minimize the variance within clusters, which is a calculation that heavily weights the centroid, ward proves to be a poor choice. Let's plot the centroids to make this a little more tangible. 

In [ ]:
# Run this cell unchanged

# Create dataframe to make filtering simpler
import pandas as pd
ward = pd.DataFrame(data_4).assign(label=preds)

# Filter dataframe according to their clusters

label_0_center = ward.query('label==0').mean()[:2] # Calculate the mean to 
label_1_center = ward.query('label==1').mean()[:2] # get the coordinates of the centroid

# Plot the results
plt.scatter(data_4[:,0], data_4[:,1], c=preds);
plt.scatter(label_0_center[0], label_0_center[1], s=100, label='Cluster 0 Centroid')
plt.scatter(label_1_center[0], label_1_center[1], s=100, label='Cluster 1 Centroid')
plt.legend(loc='upper left');

The results of ward clustering with this data are not that much different from the results produced by KMeans. Kmeans and Ward clustering are both methods that focus on minimizing variance and because of this, the structures that Kmeans and Ward fail to understand are quite similar. 

But if we apply ward clustering to the previous dataset we will see a major stength of Ward over Kmeans.

In [ ]:
# Run this cell unchanged

ward = AgglomerativeClustering(n_clusters=3, linkage='ward')
ward.fit(data_3)
preds = ward.labels_
plt.scatter(data_3[:,0], data_3[:,1], c=preds);

Above we can see that AgglomerativeClustering is much better at finding clusters when the variance of the clusters are not consistent. 

## Bonus

When it comes to the fourth dataset, there are several options for modeling the correct clusters. The first one is to use the linkage setting `'single'` with Agglomerative clustering. Single linkage creates clusters by finding the points from different clusters that are closest to each other and merging their clusters together. For the 4th dataset, because the two clusters never touch, and because distance between the clusters is much larger smaller than the points within the cluster, the clusters end up being quite perfect!

In [ ]:
# Run this cell unchanged

options = ['ward', 'complete', 'average', 'single']
fig, axes = plt.subplots(2,2, figsize=(15,6))

for idx in range(len(options)):
    ax = axes[idx//2, idx%2]
    option = options[idx]

    agg_cluster = AgglomerativeClustering(n_clusters=2, linkage=option)
    agg_cluster.fit(data_4)
    preds = agg_cluster.labels_
    ax.scatter(data_4[:,0], data_4[:,1], c=preds)
    ax.set_title(option)

Sklearn's recommended approach for this data is to set linkage to `average` and calculate the "connectivity" matrix which is just a matrix describing the similarity of each row in the dataset.

Let's look at an example of this matrix.

In [ ]:
# Run this cell unchanged

from sklearn.neighbors import kneighbors_graph

# A simple dataset
X = [[0], 
     [3], 
     [1]]

# Pass the simple dataset into kneighbors_graph
A = kneighbors_graph(X, 2, mode='connectivity', include_self=True)
# Output the connectivity matrix
A.toarray()

Above we end up with a matrix, where the first row is a lot like the last row and the second row is more like the last row than the first.

If we consider that the original rows of our data were

```
0
3
1
```

Where 0 is pretty close to 1 and 3 is closer to 1 than it is to 0, the rows of our connectivity matrix are quite sensical!

Let's create a connectivity matrix for data_4 and pass it into an Average Linking Cluster Model.

In [ ]:
# Run this cell unchanged

connectivity = kneighbors_graph(data_4, n_neighbors=10, include_self=False)

In [ ]:
# Run this cell unchanged

average = AgglomerativeClustering(linkage="average",
                                  n_clusters=2, 
                                  connectivity=connectivity)

average.fit(data_4)
preds = average.labels_
plt.scatter(data_4[:,0], data_4[:,1], c=preds);